<a href="https://colab.research.google.com/github/seoyeon0905/mimic-readmission-analysis/blob/main/notebooks/05_readmission_risk_factors.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>